In [ ]:
pip install torch matplotlib crewai crewai_tools

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 8.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.7/197.7 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.0/463.0 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.4/211.4 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# # Load environment variables
# from helper import load_env
# load_env()

import os
import yaml
from crewai import Agent, Task, Crew

In [ ]:
import os
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'
os.environ['OPENAI_API_KEY']= 'your_api_key_here'

In [ ]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

In [ ]:
from pydantic import BaseModel, Field
from typing import Dict, Optional, List, Set, Tuple
from crewai import Agent, Task, Crew, Flow
from crewai.flow.flow import listen, start

# Define file paths for YAML configurations
files = {
    'agents': '/content/config1/six_hats_agents.yaml', # your path
    'tasks': '/content/config1/six_hats_tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations
agents_config = configs['agents']
tasks_config = configs['tasks']

from pydantic import BaseModel, Field
from typing import Dict, Optional, List, Set, Tuple
from crewai import Agent, Task, Crew, Flow
from crewai.flow.flow import listen, start

# [Previous imports and configurations remain the same]

class DecisionMakingPipeline(Flow):
    @start()
    def get_decision(self):
        decision = {
            "decision_data": {
                "title": "City move decision",
                "context": "Evaluating if John has to pick up a new job in Seattle, where he is new",
                "timeframe": "Q4 2024",
                "budget": "1000000",
                "stakeholders": ["John", "Wife", "Daughter"],
                "constraints": ["New Location", "Time sensitive", "Change averse"]
            }
        }
        self.state["original_decision"] = decision
        return [decision]

    @listen(get_decision)
    def perform_hat_analysis(self, decisions):
        analyses = analysis_crew.kickoff_for_each(decisions)
        self.state["hat_analyses"] = analyses
        return analyses

    @listen(perform_hat_analysis)
    def synthesize_and_plan(self, analyses):
        synthesis_input = [{
            "analysis_results": analysis,
            "original_decision": self.state["original_decision"]
        } for analysis in analyses]

        recommendations = synthesis_crew.kickoff_for_each(synthesis_input)
        self.state["recommendations"] = recommendations
        return recommendations

    @listen(synthesize_and_plan)
    def finalize_recommendation(self, recommendations):
        final_recommendations = []
        for rec in recommendations:
            # Convert CrewOutput to dictionary and extract relevant fields
            try:
                # Access the output attribute of CrewOutput
                output_dict = rec.output
                final_rec = {
                    "summary": output_dict.get("summary", "No summary provided"),
                    "key_points": output_dict.get("key_points", []),
                    "action_plan": output_dict.get("action_plan", {}),
                    "metrics": output_dict.get("metrics", [])
                }
            except AttributeError:
                # Fallback if the structure is different
                final_rec = {
                    "summary": str(rec),
                    "key_points": [],
                    "action_plan": {},
                    "metrics": []
                }
            final_recommendations.append(final_rec)
        return final_recommendations



# Creating Agents
white_hat_agent = Agent(
    config=agents_config['white_hat_agent']
)

red_hat_agent = Agent(
    config=agents_config['red_hat_agent']
)

black_hat_agent = Agent(
    config=agents_config['black_hat_agent']
)

yellow_hat_agent = Agent(
    config=agents_config['yellow_hat_agent']
)

green_hat_agent = Agent(
    config=agents_config['green_hat_agent']
)

blue_hat_agent = Agent(
    config=agents_config['blue_hat_agent']
)

synthesis_specialist = Agent(
    config=agents_config['synthesis_specialist']
)

implementation_strategist = Agent(
    config=agents_config['implementation_strategist']
)

# Creating Tasks for Analysis Crew
white_hat_task = Task(
    config=tasks_config['white_hat_analysis'],
    agent=white_hat_agent
)

red_hat_task = Task(
    config=tasks_config['red_hat_analysis'],
    agent=red_hat_agent
)

black_hat_task = Task(
    config=tasks_config['black_hat_analysis'],
    agent=black_hat_agent
)

yellow_hat_task = Task(
    config=tasks_config['yellow_hat_analysis'],
    agent=yellow_hat_agent
)

green_hat_task = Task(
    config=tasks_config['green_hat_analysis'],
    agent=green_hat_agent
)

blue_hat_task = Task(
    config=tasks_config['blue_hat_analysis'],
    agent=blue_hat_agent
)

# Creating Analysis Crew
analysis_crew = Crew(
    agents=[
        white_hat_agent,
        red_hat_agent,
        black_hat_agent,
        yellow_hat_agent,
        green_hat_agent,
        blue_hat_agent
    ],
    tasks=[
        white_hat_task,
        red_hat_task,
        black_hat_task,
        yellow_hat_task,
        green_hat_task,
        blue_hat_task
    ],
    verbose=True
)

# Creating Tasks for Synthesis Crew
synthesis_task = Task(
    config=tasks_config['synthesis'],
    agent=synthesis_specialist
)

implementation_task = Task(
    config=tasks_config['implementation_planning'],
    agent=implementation_strategist
)

# Creating Synthesis Crew
synthesis_crew = Crew(
    agents=[
        synthesis_specialist,
        implementation_strategist
    ],
    tasks=[
        synthesis_task,
        implementation_task
    ],
    verbose=True
)

# Initialize the pipeline
flow = DecisionMakingPipeline()

# To run the pipeline:
# results = flow.kickoff()

In [ ]:
flow.plot()

Plot saved as crewai_flow.html


In [ ]:
import asyncio
# Modified to run in colab
# Instead of using asyncio.run(), get the current event loop
# and run the kickoff_async() coroutine within it.
results = await asyncio.get_event_loop().run_in_executor(None, flow.kickoff_async)
# or you could use create_task for better management
# and potential cancellation of the task
task = asyncio.create_task(flow.kickoff_async())
result = await task

# Agent: Information Gatherer
## Task: Collect and present all available data relevant to the decision using {'title': 'City move decision', 'context': 'Evaluating if John has to pick up a new job in Seattle, where he is new', 'timeframe': 'Q4 2024', 'budget': '1000000', 'stakeholders': ['John', 'Wife', 'Daughter'], 'constraints': ['New Location', 'Time sensitive', 'Change averse']}


# Agent: Information Gatherer
## Final Answer: 
**City Move Decision Report**

**Title:** City Move Decision

**Context:** Evaluating if John has to pick up a new job in Seattle, where he is new.

**Timeframe:** Q4 2024

**Budget:** $1,000,000

**Stakeholders:** 
1. John - Prospective Job Seeker
2. Wife - Potential Relocation Partner 
3. Daughter - Family Member Affected by the Move

**Constraints:**
1. **New Location:** Challenges of moving to an unfamiliar city, including understanding the lifestyle, schools, community, and amenities involved in Seattle.
2. **Time Sensitive:** A decision is required by 



# Agent: Process Controller
## Final Answer: 
**Overview of the Analysis Process:**

The decision-making process regarding the move to Seattle for John and his family has been structured around understanding their emotional and practical needs within a time-sensitive context. It recognizes the unique challenges posed by being in a new location, the family's change-averse tendencies, and the specific stakeholders involved: John, his wife, and their daughter.

**Key Points of Analysis:**

1. **Community Exploration Program**: The family will engage in a structured exploration of Seattle’s neighborhoods, emphasizing cultural and recreational activities significant to each member. A schedule will be created to ensure every family member has equal input, fostering involvement and reducing resistance to change.

2. **Educational Integration Sessions**: Before the move, efforts must focus on integrating their daughter into the Seattle community through activities and workshops. This early e

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/crewai/flow/flow.py", line 363, in _execute_single_listener
    listener_result = await self._execute_method(
  File "/usr/local/lib/python3.10/dist-packages/crewai/flow/flow.py", line 306, in _execute_method
    else method(*args, **kwargs)
  File "<ipython-input-54-82ee40d25b49>", line 58, in synthesize_and_plan
    recommendations = synthesis_crew.kickoff_for_each(synthesis_input)
  File "/usr/local/lib/python3.10/dist-packages/crewai/crew.py", line 521, in kickoff_for_each
    output = crew.kickoff(inputs=input_data)
  File "/usr/local/lib/python3.10/dist-packages/crewai/crew.py", line 468, in kickoff
    self._interpolate_inputs(inputs)
  File "/usr/local/lib/python3.10/dist-packages/crewai/crew.py", line 951, in _interpolate_inputs
    [
  File "/usr/local/lib/python3.10/dist-packages/crewai/crew.py", line 952, in <listcomp>
    task.interpolate_inputs(
  File "/usr/local/lib/python3.10/dist-packag

In [ ]:
print (result)

[CrewOutput(raw='**Overview of the Analysis Process:**\n\nThe decision-making process regarding the move to Seattle for John and his family has been structured around understanding their emotional and practical needs within a time-sensitive context. It recognizes the unique challenges posed by being in a new location, the family\'s change-averse tendencies, and the specific stakeholders involved: John, his wife, and their daughter.\n\n**Key Points of Analysis:**\n\n1. **Community Exploration Program**: The family will engage in a structured exploration of Seattle’s neighborhoods, emphasizing cultural and recreational activities significant to each member. A schedule will be created to ensure every family member has equal input, fostering involvement and reducing resistance to change.\n\n2. **Educational Integration Sessions**: Before the move, efforts must focus on integrating their daughter into the Seattle community through activities and workshops. This early engagement will support

In [ ]:
# Print and save: One version
def save_decision_results(results, filename='decision_analysis.md'):
    """
    Takes the flow results and saves them to a markdown file
    """
    try:
        formatted_content = results[0]['summary'] if results and isinstance(results, list) else results

        # Write to file
        with open(filename, 'w') as f:
            f.write(formatted_content)

        print(f"Results have been saved to {filename}")

    except Exception as e:
        print(f"Error saving results: {str(e)}")

# After running your flow, use it like this:

save_decision_results(result)

Error saving results: "Key 'summary' not found in CrewOutput."


In [ ]:
# Print and Save when the above doesn't work
def save_decision_results(results, filename='decision_analysis.md'):
    """
    Takes the flow results and saves them to a markdown file, handling CrewOutput format
    """
    try:
        if isinstance(results, list) and results:
            crew_output = results[0]  # Get the first CrewOutput object

            # Extract the raw content from CrewOutput
            if hasattr(crew_output, 'raw'):
                content = crew_output.raw
            else:
                # Fallback in case structure is different
                content = str(crew_output)

            # Write to file
            with open(filename, 'w') as f:
                f.write(content)

            print(f"Results have been saved to {filename}")

            # Print formatted version to console
            print("\n" + "="*80)
            print("Decision Analysis Results")
            print("="*80 + "\n")
            print(content)
            print("\n" + "="*80)

    except Exception as e:
        print(f"Error saving results: {str(e)}")

# To use after running your flow:
# flow = DecisionMakingPipeline()
# results = flow.kickoff()
save_decision_results(result)

Results have been saved to decision_analysis.md

Decision Analysis Results

**Overview of the Analysis Process:**

The decision-making process regarding the move to Seattle for John and his family has been structured around understanding their emotional and practical needs within a time-sensitive context. It recognizes the unique challenges posed by being in a new location, the family's change-averse tendencies, and the specific stakeholders involved: John, his wife, and their daughter.

**Key Points of Analysis:**

1. **Community Exploration Program**: The family will engage in a structured exploration of Seattle’s neighborhoods, emphasizing cultural and recreational activities significant to each member. A schedule will be created to ensure every family member has equal input, fostering involvement and reducing resistance to change.

2. **Educational Integration Sessions**: Before the move, efforts must focus on integrating their daughter into the Seattle community through activities

In [ ]:
# six_hats_agents.yaml

white_hat_agent:
  role: Information Gatherer
  goal: Gather all relevant facts and data related to the decision
  backstory: Focuses on neutral and objective data gathering
  verbose: true
  allow_delegation: false

red_hat_agent:
  role: Emotional Evaluator
  goal: Assess emotional responses and intuitive reactions to the decision
  backstory: Specializes in understanding feelings and emotional impacts
  verbose: true
  allow_delegation: false

black_hat_agent:
  role: Critical Analyzer
  goal: Identify potential risks, problems, and weaknesses
  backstory: Expert in spotting potential difficulties and challenges
  verbose: true
  allow_delegation: false

yellow_hat_agent:
  role: Benefit Finder
  goal: Identify advantages and opportunities in the decision
  backstory: Focuses on finding positive aspects and potential benefits
  verbose: true
  allow_delegation: false

green_hat_agent:
  role: Creative Thinker
  goal: Generate alternative solutions and creative approaches
  backstory: Specializes in innovative thinking and new possibilities
  verbose: true
  allow_delegation: false

blue_hat_agent:
  role: Process Controller
  goal: Manage the thinking process and organize conclusions
  backstory: Orchestrates the thinking process and maintains focus
  verbose: true
  allow_delegation: false

synthesis_specialist:
  role: Integration Expert
  goal: Synthesize insights from all hat analyses into coherent recommendations
  backstory: Expert in combining diverse perspectives into actionable insights
  verbose: true
  allow_delegation: false

implementation_strategist:
  role: Action Planner
  goal: Develop concrete implementation plans from recommendations
  backstory: Specializes in turning insights into practical action steps
  verbose: true
  allow_delegation: false



In [ ]:
# six_hats_tasks.yaml
white_hat_analysis:
  description: Collect and present all available data relevant to the decision using {decision_data}
  expected_output: A report summarizing facts and data
  verbose: true

red_hat_analysis:
  description: Evaluate emotional aspects and intuitive reactions to the decision using {decision_data}
  expected_output: A summary of emotional impacts and stakeholder feelings
  verbose: true

black_hat_analysis:
  description: Analyze potential risks and problems in the decision using {decision_data}
  expected_output: A comprehensive risk assessment report
  verbose: true

yellow_hat_analysis:
  description: Identify benefits and opportunities in the decision using {decision_data}
  expected_output: A summary of advantages and potential positive outcomes
  verbose: true

green_hat_analysis:
  description: Generate creative solutions and alternative approaches
  expected_output: A list of innovative ideas and solutions
  verbose: true

blue_hat_analysis:
  description: Coordinate the thinking process and synthesize initial findings
  expected_output: A structured overview of the analysis process and key points
  verbose: true

synthesis:
  description: Integrate insights from all hat analyses into recommendations
  expected_output: A comprehensive synthesis of all perspectives and key recommendations
  verbose: true

implementation_planning:
  description: Develop concrete action plans from synthesized recommendations and provides synthesis as appendix.
  expected_output: A detailed implementation plan with timelines and responsibilities
  verbose: true